# Experiment 031: jiweiliu Fast SA with Deletion Cascade

Implementing the jiweiliu kernel's approach:
1. Generate grid configurations with varying ncols, nrows, append_x, append_y
2. Run SA optimization on each configuration
3. Apply deletion cascade - propagate good large configs to smaller N
4. This generates FUNDAMENTALLY DIFFERENT solutions

In [ ]:
import math
import os
import time
from multiprocessing import Pool, cpu_count

import numpy as np
import pandas as pd
from numba import njit
from numba.typed import List as NumbaList
from shapely.geometry import Polygon
from shapely.affinity import rotate, translate

print(f"CPU count: {cpu_count()}")

In [ ]:
# Tree shape constants (must match official spec)
TRUNK_W = 0.15
TRUNK_H = 0.2
BASE_W = 0.7
MID_W = 0.4
TOP_W = 0.25
TIP_Y = 0.8
TIER_1_Y = 0.5
TIER_2_Y = 0.25
BASE_Y = 0.0
TRUNK_BOTTOM_Y = -TRUNK_H

# Maximum distance between tree centers for possible overlap
MAX_OVERLAP_DIST = 1.8
MAX_OVERLAP_DIST_SQ = MAX_OVERLAP_DIST * MAX_OVERLAP_DIST

# Tree vertices in canonical position
TREE_VERTICES = np.array([
    [0.0, TIP_Y],
    [TOP_W / 2.0, TIER_1_Y],
    [TOP_W / 4.0, TIER_1_Y],
    [MID_W / 2.0, TIER_2_Y],
    [MID_W / 4.0, TIER_2_Y],
    [BASE_W / 2.0, BASE_Y],
    [TRUNK_W / 2.0, BASE_Y],
    [TRUNK_W / 2.0, TRUNK_BOTTOM_Y],
    [-TRUNK_W / 2.0, TRUNK_BOTTOM_Y],
    [-TRUNK_W / 2.0, BASE_Y],
    [-BASE_W / 2.0, BASE_Y],
    [-MID_W / 4.0, TIER_2_Y],
    [-MID_W / 2.0, TIER_2_Y],
    [-TOP_W / 4.0, TIER_1_Y],
    [-TOP_W / 2.0, TIER_1_Y],
], dtype=np.float64)

print(f"Tree vertices: {len(TREE_VERTICES)} points")

In [ ]:
@njit(cache=True)
def rotate_point(x, y, cos_a, sin_a):
    return x * cos_a - y * sin_a, x * sin_a + y * cos_a


@njit(cache=True)
def get_tree_vertices(cx, cy, angle_deg):
    """Get 15 vertices of tree polygon at given position and angle."""
    angle_rad = angle_deg * math.pi / 180.0
    cos_a = math.cos(angle_rad)
    sin_a = math.sin(angle_rad)
    vertices = np.empty((15, 2), dtype=np.float64)
    pts = np.array([
        [0.0, 0.8],
        [0.125, 0.5],
        [0.0625, 0.5],
        [0.2, 0.25],
        [0.1, 0.25],
        [0.35, 0.0],
        [0.075, 0.0],
        [0.075, -0.2],
        [-0.075, -0.2],
        [-0.075, 0.0],
        [-0.35, 0.0],
        [-0.1, 0.25],
        [-0.2, 0.25],
        [-0.0625, 0.5],
        [-0.125, 0.5],
    ], dtype=np.float64)
    for i in range(15):
        rx, ry = rotate_point(pts[i, 0], pts[i, 1], cos_a, sin_a)
        vertices[i, 0] = rx + cx
        vertices[i, 1] = ry + cy
    return vertices

In [ ]:
@njit(cache=True)
def polygon_bounds(vertices):
    """Get bounding box of polygon vertices."""
    min_x = vertices[0, 0]
    min_y = vertices[0, 1]
    max_x = vertices[0, 0]
    max_y = vertices[0, 1]
    for i in range(1, vertices.shape[0]):
        x = vertices[i, 0]
        y = vertices[i, 1]
        if x < min_x:
            min_x = x
        if x > max_x:
            max_x = x
        if y < min_y:
            min_y = y
        if y > max_y:
            max_y = y
    return min_x, min_y, max_x, max_y


@njit(cache=True)
def point_in_polygon(px, py, vertices):
    """Check if point is inside polygon using ray casting."""
    n = vertices.shape[0]
    inside = False
    j = n - 1
    for i in range(n):
        xi, yi = vertices[i, 0], vertices[i, 1]
        xj, yj = vertices[j, 0], vertices[j, 1]
        if ((yi > py) != (yj > py)) and (px < (xj - xi) * (py - yi) / (yj - yi) + xi):
            inside = not inside
        j = i
    return inside


@njit(cache=True)
def segments_intersect(p1x, p1y, p2x, p2y, p3x, p3y, p4x, p4y):
    """Check if two line segments intersect."""
    dax = p2x - p1x
    day = p2y - p1y
    dbx = p4x - p3x
    dby = p4y - p3y

    d1x = p1x - p3x
    d1y = p1y - p3y
    d2x = p2x - p3x
    d2y = p2y - p3y

    cross_b1 = dbx * d1y - dby * d1x
    cross_b2 = dbx * d2y - dby * d2x

    if cross_b1 * cross_b2 > 0:
        return False

    d3x = p3x - p1x
    d3y = p3y - p1y
    d4x = p4x - p1x
    d4y = p4y - p1y

    cross_a1 = dax * d3y - day * d3x
    cross_a2 = dax * d4y - day * d4x

    if cross_a1 * cross_a2 > 0:
        return False

    return True

In [ ]:
@njit(cache=True)
def polygons_overlap(verts1, verts2, cx1, cy1, cx2, cy2):
    """Check if two polygons overlap (not just touch)."""
    # Quick center distance check
    dx = cx2 - cx1
    dy = cy2 - cy1
    dist_sq = dx * dx + dy * dy
    if dist_sq > MAX_OVERLAP_DIST_SQ:
        return False

    # Bounding box check
    min_x1, min_y1, max_x1, max_y1 = polygon_bounds(verts1)
    min_x2, min_y2, max_x2, max_y2 = polygon_bounds(verts2)
    if max_x1 < min_x2 or max_x2 < min_x1 or max_y1 < min_y2 or max_y2 < min_y1:
        return False

    # Check if any vertex of poly1 is inside poly2
    for i in range(verts1.shape[0]):
        if point_in_polygon(verts1[i, 0], verts1[i, 1], verts2):
            return True

    # Check if any vertex of poly2 is inside poly1
    for i in range(verts2.shape[0]):
        if point_in_polygon(verts2[i, 0], verts2[i, 1], verts1):
            return True

    # Check edge intersections
    n1 = verts1.shape[0]
    n2 = verts2.shape[0]
    for i in range(n1):
        j = (i + 1) % n1
        p1x, p1y = verts1[i, 0], verts1[i, 1]
        p2x, p2y = verts1[j, 0], verts1[j, 1]
        for k in range(n2):
            m = (k + 1) % n2
            p3x, p3y = verts2[k, 0], verts2[k, 1]
            p4x, p4y = verts2[m, 0], verts2[m, 1]
            if segments_intersect(p1x, p1y, p2x, p2y, p3x, p3y, p4x, p4y):
                return True
    return False


@njit(cache=True)
def has_any_overlap(all_vertices, centers_x, centers_y):
    """Check if any pair of polygons overlap."""
    n = len(all_vertices)
    for i in range(n):
        for j in range(i + 1, n):
            if polygons_overlap(all_vertices[i], all_vertices[j],
                              centers_x[i], centers_y[i], centers_x[j], centers_y[j]):
                return True
    return False

In [ ]:
@njit(cache=True)
def compute_bounding_box(all_vertices):
    """Compute overall bounding box of all polygons."""
    min_x = math.inf
    min_y = math.inf
    max_x = -math.inf
    max_y = -math.inf
    for verts in all_vertices:
        x1, y1, x2, y2 = polygon_bounds(verts)
        if x1 < min_x:
            min_x = x1
        if y1 < min_y:
            min_y = y1
        if x2 > max_x:
            max_x = x2
        if y2 > max_y:
            max_y = y2
    return min_x, min_y, max_x, max_y


@njit(cache=True)
def get_side_length(all_vertices):
    """Get side length of bounding square."""
    min_x, min_y, max_x, max_y = compute_bounding_box(all_vertices)
    return max(max_x - min_x, max_y - min_y)


@njit(cache=True)
def calculate_score_numba(all_vertices):
    """Calculate score = max(width, height)^2 / n"""
    side = get_side_length(all_vertices)
    return side * side / len(all_vertices)

In [ ]:
@njit(cache=True)
def create_grid_vertices_extended(seed_xs, seed_ys, seed_degs, a, b, ncols, nrows, append_x, append_y):
    """
    Create grid of tree vertices by translation with optional append.
    Returns both vertices and center coordinates.
    """
    n_seeds = len(seed_xs)

    # Calculate total number of trees
    n_base = n_seeds * ncols * nrows
    n_append_x = nrows if append_x else 0
    n_append_y = ncols if append_y else 0
    n_total = n_base + n_append_x + n_append_y

    all_vertices = []
    centers_x = np.empty(n_total, dtype=np.float64)
    centers_y = np.empty(n_total, dtype=np.float64)

    idx = 0
    # Base grid
    for s in range(n_seeds):
        for col in range(ncols):
            for row in range(nrows):
                cx = seed_xs[s] + col * a
                cy = seed_ys[s] + row * b
                all_vertices.append(get_tree_vertices(cx, cy, seed_degs[s]))
                centers_x[idx] = cx
                centers_y[idx] = cy
                idx += 1

    # Append in x direction
    if append_x and n_seeds > 1:
        for row in range(nrows):
            cx = seed_xs[1] + ncols * a
            cy = seed_ys[1] + row * b
            all_vertices.append(get_tree_vertices(cx, cy, seed_degs[1]))
            centers_x[idx] = cx
            centers_y[idx] = cy
            idx += 1

    # Append in y direction
    if append_y and n_seeds > 1:
        for col in range(ncols):
            cx = seed_xs[1] + col * a
            cy = seed_ys[1] + nrows * b
            all_vertices.append(get_tree_vertices(cx, cy, seed_degs[1]))
            centers_x[idx] = cx
            centers_y[idx] = cy
            idx += 1

    return all_vertices, centers_x, centers_y

In [ ]:
@njit(cache=True)
def get_initial_translations(seed_xs, seed_ys, seed_degs):
    """Get initial translation lengths from seed configuration."""
    n_seeds = len(seed_xs)
    if n_seeds < 2:
        return 1.0, 1.0
    
    # Calculate distances between seeds
    dx = seed_xs[1] - seed_xs[0]
    dy = seed_ys[1] - seed_ys[0]
    
    # Use the distance as initial translation
    a = abs(dx) if abs(dx) > 0.1 else 0.9
    b = abs(dy) if abs(dy) > 0.1 else 0.75
    
    return a, b

In [ ]:
@njit(cache=True)
def sa_optimize_improved(
    seed_xs_init,
    seed_ys_init,
    seed_degs_init,
    a_init,
    b_init,
    ncols,
    nrows,
    append_x,
    append_y,
    Tmax,
    Tmin,
    nsteps,
    nsteps_per_T,
    position_delta,
    angle_delta,
    angle_delta2,
    delta_t,
    random_seed,
):
    """
    Improved simulated annealing with:
    1. Translation lengths optimized via SA
    2. rotate_all move type
    3. append_x/append_y support
    """
    np.random.seed(random_seed)
    n_seeds = len(seed_xs_init)

    # Copy initial seeds
    seed_xs = seed_xs_init.copy()
    seed_ys = seed_ys_init.copy()
    seed_degs = seed_degs_init.copy()

    # Initial translations
    a = a_init
    b = b_init

    # Create initial grid and check validity
    all_vertices, centers_x, centers_y = create_grid_vertices_extended(
        seed_xs, seed_ys, seed_degs, a, b, ncols, nrows, append_x, append_y
    )
    
    if has_any_overlap(all_vertices, centers_x, centers_y):
        # Try to find valid initial translations
        a_test, b_test = get_initial_translations(seed_xs, seed_ys, seed_degs)
        a = max(a, a_test * 1.5)
        b = max(b, b_test * 1.5)
        all_vertices, centers_x, centers_y = create_grid_vertices_extended(
            seed_xs, seed_ys, seed_degs, a, b, ncols, nrows, append_x, append_y
        )
        
        if has_any_overlap(all_vertices, centers_x, centers_y):
            # Still overlapping, return large score
            return 1e10, seed_xs, seed_ys, seed_degs, a, b

    current_score = calculate_score_numba(all_vertices)
    best_score = current_score
    best_seed_xs = seed_xs.copy()
    best_seed_ys = seed_ys.copy()
    best_seed_degs = seed_degs.copy()
    best_a = a
    best_b = b

    # Temperature schedule
    T = Tmax
    alpha = (Tmin / Tmax) ** (1.0 / nsteps)

    for step in range(nsteps):
        for _ in range(nsteps_per_T):
            # Choose move type
            move_type = np.random.randint(0, 6)
            
            # Save current state
            old_seed_xs = seed_xs.copy()
            old_seed_ys = seed_ys.copy()
            old_seed_degs = seed_degs.copy()
            old_a = a
            old_b = b

            if move_type == 0:  # Move single seed position
                idx = np.random.randint(0, n_seeds)
                seed_xs[idx] += (np.random.random() - 0.5) * 2 * position_delta
                seed_ys[idx] += (np.random.random() - 0.5) * 2 * position_delta
            elif move_type == 1:  # Rotate single seed
                idx = np.random.randint(0, n_seeds)
                seed_degs[idx] += (np.random.random() - 0.5) * 2 * angle_delta
            elif move_type == 2:  # Rotate all seeds
                delta = (np.random.random() - 0.5) * 2 * angle_delta2
                for i in range(n_seeds):
                    seed_degs[i] += delta
            elif move_type == 3:  # Change translation a
                a += (np.random.random() - 0.5) * 2 * delta_t
                a = max(0.5, a)
            elif move_type == 4:  # Change translation b
                b += (np.random.random() - 0.5) * 2 * delta_t
                b = max(0.5, b)
            else:  # Move all seeds
                dx = (np.random.random() - 0.5) * 2 * position_delta
                dy = (np.random.random() - 0.5) * 2 * position_delta
                for i in range(n_seeds):
                    seed_xs[i] += dx
                    seed_ys[i] += dy

            # Create new grid
            all_vertices, centers_x, centers_y = create_grid_vertices_extended(
                seed_xs, seed_ys, seed_degs, a, b, ncols, nrows, append_x, append_y
            )

            # Check validity
            if has_any_overlap(all_vertices, centers_x, centers_y):
                # Reject move
                seed_xs = old_seed_xs
                seed_ys = old_seed_ys
                seed_degs = old_seed_degs
                a = old_a
                b = old_b
                continue

            new_score = calculate_score_numba(all_vertices)
            delta_score = new_score - current_score

            # Accept or reject
            if delta_score < 0 or np.random.random() < math.exp(-delta_score / T):
                current_score = new_score
                if new_score < best_score:
                    best_score = new_score
                    best_seed_xs = seed_xs.copy()
                    best_seed_ys = seed_ys.copy()
                    best_seed_degs = seed_degs.copy()
                    best_a = a
                    best_b = b
            else:
                # Reject move
                seed_xs = old_seed_xs
                seed_ys = old_seed_ys
                seed_degs = old_seed_degs
                a = old_a
                b = old_b

        T *= alpha

    return best_score, best_seed_xs, best_seed_ys, best_seed_degs, best_a, best_b

In [ ]:
def optimize_grid_config(args):
    """Wrapper for parallel execution."""
    ncols, nrows, append_x, append_y, initial_seeds, a_init, b_init, sa_params, seed = args
    
    seed_xs = np.array([s[0] for s in initial_seeds], dtype=np.float64)
    seed_ys = np.array([s[1] for s in initial_seeds], dtype=np.float64)
    seed_degs = np.array([s[2] for s in initial_seeds], dtype=np.float64)
    
    score, opt_xs, opt_ys, opt_degs, opt_a, opt_b = sa_optimize_improved(
        seed_xs, seed_ys, seed_degs,
        a_init, b_init,
        ncols, nrows, append_x, append_y,
        sa_params["Tmax"],
        sa_params["Tmin"],
        sa_params["nsteps"],
        sa_params["nsteps_per_T"],
        sa_params["position_delta"],
        sa_params["angle_delta"],
        sa_params["angle_delta2"],
        sa_params["delta_t"],
        seed,
    )
    
    # Calculate number of trees
    n_seeds = len(initial_seeds)
    n_base = n_seeds * ncols * nrows
    n_append_x = nrows if append_x else 0
    n_append_y = ncols if append_y else 0
    n_trees = n_base + n_append_x + n_append_y
    
    # Generate tree data
    tree_data = []
    for s in range(n_seeds):
        for col in range(ncols):
            for row in range(nrows):
                cx = opt_xs[s] + col * opt_a
                cy = opt_ys[s] + row * opt_b
                tree_data.append((cx, cy, opt_degs[s]))
    
    if append_x and n_seeds > 1:
        for row in range(nrows):
            cx = opt_xs[1] + ncols * opt_a
            cy = opt_ys[1] + row * opt_b
            tree_data.append((cx, cy, opt_degs[1]))
    
    if append_y and n_seeds > 1:
        for col in range(ncols):
            cx = opt_xs[1] + col * opt_a
            cy = opt_ys[1] + nrows * opt_b
            tree_data.append((cx, cy, opt_degs[1]))
    
    return n_trees, score, tree_data

In [ ]:
def load_submission_data(filepath):
    """Load submission and return flattened arrays."""
    df = pd.read_csv(filepath)

    all_xs = []
    all_ys = []
    all_degs = []

    for n in range(1, 201):
        prefix = f"{n:03d}_"
        group = df[df["id"].str.startswith(prefix)].sort_values("id")
        for _, row in group.iterrows():
            x = float(str(row["x"])[1:]) if str(row["x"]).startswith('s') else float(row["x"])
            y = float(str(row["y"])[1:]) if str(row["y"]).startswith('s') else float(row["y"])
            deg = float(str(row["deg"])[1:]) if str(row["deg"]).startswith('s') else float(row["deg"])
            all_xs.append(x)
            all_ys.append(y)
            all_degs.append(deg)

    return np.array(all_xs), np.array(all_ys), np.array(all_degs)


def save_submission(filepath, all_xs, all_ys, all_degs):
    """Save submission from flattened arrays."""
    rows = []
    idx = 0
    for n in range(1, 201):
        for t in range(n):
            rows.append({
                "id": f"{n:03d}_{t}",
                "x": f"s{all_xs[idx]}",
                "y": f"s{all_ys[idx]}",
                "deg": f"s{all_degs[idx]}",
            })
            idx += 1

    df = pd.DataFrame(rows)
    df.to_csv(filepath, index=False)


def calculate_total_score(all_xs, all_ys, all_degs):
    """Calculate total score across all groups."""
    total = 0.0
    idx = 0
    for n in range(1, 201):
        vertices = [get_tree_vertices(all_xs[idx + i], all_ys[idx + i], all_degs[idx + i]) for i in range(n)]
        score = calculate_score_numba(vertices)
        total += score
        idx += n
    return total

In [ ]:
# Load baseline
baseline_path = '/home/submission/submission.csv'
print(f"Loading baseline: {baseline_path}")
baseline_xs, baseline_ys, baseline_degs = load_submission_data(baseline_path)
baseline_total = calculate_total_score(baseline_xs, baseline_ys, baseline_degs)
print(f"Baseline total score: {baseline_total:.6f}")

In [ ]:
# Initial 2-tree seed configuration (from jiweiliu kernel)
initial_seeds = [
    (-4.191683864412409, -4.498489528496051, 74.54421568660419),
    (-4.92202045352307, -4.727639556649786, 254.5401905706735),
]

# Initial translation lengths
a_init = 0.8744896974945239
b_init = 0.7499641699190263

print(f"Initial seeds: {len(initial_seeds)}")
print(f"Initial translations: a={a_init:.4f}, b={b_init:.4f}")

In [ ]:
# Grid configurations: (ncols, nrows, append_x, append_y)
grid_configs = []

# Generate configurations for better coverage
for ncols in range(2, 11):
    for nrows in range(ncols, 15):
        n_trees = 2 * ncols * nrows
        if 20 <= n_trees <= 200:
            grid_configs.append((ncols, nrows, False, False))
            n_with_append_y = n_trees + ncols
            if n_with_append_y <= 200:
                grid_configs.append((ncols, nrows, False, True))
            n_with_append_x = n_trees + nrows
            if n_with_append_x <= 200:
                grid_configs.append((ncols, nrows, True, False))

# Remove duplicates and sort
grid_configs = list(set(grid_configs))
grid_configs.sort(key=lambda x: (2 * x[0] * x[1] + (x[1] if x[2] else 0) + (x[0] if x[3] else 0)))

print(f"Generated {len(grid_configs)} grid configurations")
print(f"Tree counts range: {min(2*c[0]*c[1] for c in grid_configs)} to {max(2*c[0]*c[1] + (c[1] if c[2] else 0) + (c[0] if c[3] else 0) for c in grid_configs)}")

In [ ]:
# SA parameters
sa_params = {
    "Tmax": 0.001,
    "Tmin": 0.000001,
    "nsteps": 10,
    "nsteps_per_T": 10000,
    "position_delta": 0.002,
    "angle_delta": 1.0,
    "angle_delta2": 1.0,
    "delta_t": 0.002,
}

print("SA parameters:")
for k, v in sa_params.items():
    print(f"  {k}: {v}")

In [ ]:
# Warm up numba
print("Compiling numba functions...")
t0 = time.time()
dummy_xs = np.array([0.0, 1.0], dtype=np.float64)
dummy_ys = np.array([0.0, 0.0], dtype=np.float64)
dummy_degs = np.array([0.0, 180.0], dtype=np.float64)
_ = sa_optimize_improved(
    dummy_xs, dummy_ys, dummy_degs,
    1.0, 1.0, 2, 2, False, False,
    0.001, 0.0001, 2, 10,
    0.01, 10.0, 10.0, 0.01, 42
)
print(f"Compilation done in {time.time() - t0:.1f}s")

In [ ]:
# Prepare tasks
tasks = []
tree_counts = []
for i, (ncols, nrows, append_x, append_y) in enumerate(grid_configs):
    n_base = 2 * ncols * nrows
    n_append_x = nrows if append_x else 0
    n_append_y = ncols if append_y else 0
    n_trees = n_base + n_append_x + n_append_y

    if n_trees > 200:
        continue

    seed = 42 + i * 1000
    tasks.append((ncols, nrows, append_x, append_y, initial_seeds, a_init, b_init, sa_params, seed))
    tree_counts.append(n_trees)

print(f"Tasks: {len(tasks)}, tree counts: {min(tree_counts)} to {max(tree_counts)}")

In [ ]:
# Run SA optimization in parallel
print(f"Running SA optimization on {len(tasks)} configurations...")
num_workers = min(cpu_count(), len(tasks))
print(f"Using {num_workers} workers")

t0 = time.time()
with Pool(num_workers) as pool:
    results = pool.map(optimize_grid_config, tasks)
elapsed = time.time() - t0
print(f"SA optimization completed in {elapsed:.1f}s")

In [ ]:
# Collect results and compare with baseline
new_trees = {}
improved_count = 0
for n_trees, score, tree_data in results:
    if score > 1e9:  # Invalid configuration
        continue
        
    # Get baseline score for this n
    idx = sum(range(1, n_trees))
    baseline_vertices = [get_tree_vertices(baseline_xs[idx + i], baseline_ys[idx + i], baseline_degs[idx + i]) for i in range(n_trees)]
    baseline_score = calculate_score_numba(baseline_vertices)

    if score < baseline_score:
        new_trees[n_trees] = tree_data
        improvement = baseline_score - score
        if improvement > 1e-6:
            improved_count += 1
            print(f"  n={n_trees}: {score:.6f} (baseline: {baseline_score:.6f}, improved by {improvement:.6f})")

print(f"\nSA improved {improved_count} configurations")

In [ ]:
# Merge with baseline
print("Merging with baseline...")
merged_xs = baseline_xs.copy()
merged_ys = baseline_ys.copy()
merged_degs = baseline_degs.copy()

for n_trees, tree_data in new_trees.items():
    idx = sum(range(1, n_trees))
    for i in range(n_trees):
        merged_xs[idx + i] = tree_data[i][0]
        merged_ys[idx + i] = tree_data[i][1]
        merged_degs[idx + i] = tree_data[i][2]

pre_cascade_score = calculate_total_score(merged_xs, merged_ys, merged_degs)
print(f"Score after SA merge: {pre_cascade_score:.6f}")

In [ ]:
# Deletion cascade implementation
@njit(cache=True)
def deletion_cascade_numba(all_xs, all_ys, all_degs):
    """
    Apply tree deletion cascade using numba.
    For each N from 200 down to 2, check if removing any tree from N
    gives a better score for N-1 than the current N-1 solution.
    """
    # Build index mapping
    group_start = np.zeros(201, dtype=np.int64)
    for n in range(1, 201):
        group_start[n] = group_start[n-1] + (n - 1) if n > 1 else 0

    # Copy arrays
    new_xs = all_xs.copy()
    new_ys = all_ys.copy()
    new_degs = all_degs.copy()

    # Calculate initial side lengths
    side_lengths = np.zeros(201, dtype=np.float64)
    for n in range(1, 201):
        start = group_start[n]
        end = start + n
        vertices = [get_tree_vertices(new_xs[i], new_ys[i], new_degs[i]) for i in range(start, end)]
        side_lengths[n] = get_side_length(vertices)

    # Cascade from n=200 down to n=2
    for n in range(200, 1, -1):
        start_n = group_start[n]
        end_n = start_n + n
        start_prev = group_start[n - 1]

        best_side = side_lengths[n - 1]
        best_delete_idx = -1

        for del_idx in range(n):
            vertices = []
            for i in range(n):
                if i != del_idx:
                    idx = start_n + i
                    vertices.append(get_tree_vertices(new_xs[idx], new_ys[idx], new_degs[idx]))

            candidate_side = get_side_length(vertices)
            if candidate_side < best_side:
                best_side = candidate_side
                best_delete_idx = del_idx

        if best_delete_idx >= 0:
            out_idx = start_prev
            for i in range(n):
                if i != best_delete_idx:
                    in_idx = start_n + i
                    new_xs[out_idx] = new_xs[in_idx]
                    new_ys[out_idx] = new_ys[in_idx]
                    new_degs[out_idx] = new_degs[in_idx]
                    out_idx += 1
            side_lengths[n - 1] = best_side

    return new_xs, new_ys, new_degs, side_lengths

In [ ]:
# Apply tree deletion cascade
print("Applying tree deletion cascade...")
t0 = time.time()
final_xs, final_ys, final_degs, side_lengths = deletion_cascade_numba(
    merged_xs, merged_ys, merged_degs
)
print(f"Cascade completed in {time.time() - t0:.1f}s")

final_score = calculate_total_score(final_xs, final_ys, final_degs)
print(f"Final score after cascade: {final_score:.6f}")

In [ ]:
print("=" * 80)
print("Summary:")
print(f"  Baseline total:      {baseline_total:.6f}")
print(f"  After SA:            {pre_cascade_score:.6f}")
print(f"  After cascade:       {final_score:.6f}")
print(f"  Total improvement:   {baseline_total - final_score:+.6f}")
print("=" * 80)

In [ ]:
# Validate with Shapely
print("\nValidating with Shapely...")

def create_tree_polygon(x, y, deg):
    """Create a Shapely polygon for a tree."""
    tree = Polygon(TREE_VERTICES)
    tree = rotate(tree, deg, origin=(0, 0))
    tree = translate(tree, x, y)
    return tree

def validate_group(xs, ys, degs):
    """Check if any trees in a group overlap."""
    n = len(xs)
    trees = [create_tree_polygon(xs[i], ys[i], degs[i]) for i in range(n)]
    for i in range(n):
        for j in range(i + 1, n):
            if trees[i].overlaps(trees[j]) or trees[i].contains(trees[j]) or trees[j].contains(trees[i]):
                return False
    return True

# Validate all groups
overlap_count = 0
idx = 0
for n in range(1, 201):
    xs = final_xs[idx:idx+n]
    ys = final_ys[idx:idx+n]
    degs = final_degs[idx:idx+n]
    if not validate_group(xs, ys, degs):
        overlap_count += 1
        print(f"  Group {n:03d} has overlaps!")
    idx += n

if overlap_count == 0:
    print("All groups valid - no overlaps!")
else:
    print(f"\nWARNING: {overlap_count} groups have overlaps!")

In [ ]:
# Save submission if improved and valid
if final_score < baseline_total and overlap_count == 0:
    output_path = "/home/submission/submission.csv"
    save_submission(output_path, final_xs, final_ys, final_degs)
    print(f"\nSaved improved submission to {output_path}")
    print(f"New total score: {final_score:.9f}")
else:
    print(f"\nNo improvement or invalid - keeping baseline")
    print(f"Baseline score: {baseline_total:.9f}")
    final_score = baseline_total

In [ ]:
# Save metrics
import json

metrics = {
    'cv_score': final_score,
    'baseline_score': baseline_total,
    'improvement': baseline_total - final_score,
    'sa_improved_configs': improved_count,
    'overlap_count': overlap_count,
    'approach': 'jiweiliu Fast SA with Deletion Cascade'
}

with open('/home/code/experiments/031_jiweiliu_deletion_cascade/metrics.json', 'w') as f:
    json.dump(metrics, f, indent=2)

print("\nMetrics saved:")
for k, v in metrics.items():
    print(f"  {k}: {v}")